In [31]:
import lasio
import pandas as pd

In [32]:
import lasio
import pandas as pd

def stitch_las(las_path1, las_path2, output_path="stitched_output.las"):
    """
    Stitches two LAS files end-to-end based on depth index.
    Keeps the original depth indexes from both files (no merge).
    """
    # Read LAS files
    las1 = lasio.read(las_path1)
    las2 = lasio.read(las_path2)

    # Convert to DataFrames with depth as index
    df1 = las1.df()   # indexed by depth already
    df2 = las2.df()

    # Concatenate along depth index (stack)
    df_stitched = pd.concat([df1, df2])

    # Remove duplicate depths if overlap exists (keep first)
    df_stitched = df_stitched[~df_stitched.index.duplicated(keep="first")]

    # Sort by depth
    df_stitched = df_stitched.sort_index()

    # Build LAS from stitched DataFrame
    las_out = lasio.LASFile()
    las_out.set_data_from_df(df_stitched)

    # Copy well header info
    las_out.well = las1.well

    # Write LAS
    las_out.write(output_path, version=2.0)
    print(f"✅ LAS stitched and saved to {output_path}")
    print(f"Depth range: {df_stitched.index.min()} – {df_stitched.index.max()}")

    return las_out, df_stitched


In [33]:
las_file1 = "FALKON-1_MM_23 carbonate.las"
las_file2 = "FALKON-1_MM_23 clastic.las"

df_concat = stitch_las(las_file1, las_file2, output_path="FALKON-1_MM_23_Merged.las")

✅ LAS stitched and saved to FALKON-1_MM_23_Merged.las
Depth range: 1665.3 – 3824.9724
